Trying the colab based model using sparsed matrix format

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
anime_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Machine Learning/Anime Recommendation Engine/Anime Datasets/Animes2023.csv')
user_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Machine Learning/Anime Recommendation Engine/Anime Datasets/Users2023.csv')
score_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Machine Learning/Anime Recommendation Engine/Anime Datasets/UsersScores2023.csv')

In [ ]:
# Filter expert users
user_df = user_df[(user_df['Days Watched'] >= 300)]
user_df = user_df[(user_df['Mean Score'] <= 9.5) & (user_df['Mean Score'] >= 5)]
user_df = user_df[(user_df['Completed'] >= 200)]
user_df = user_df[(user_df['Episodes Watched'] >= 1000)]

In [ ]:
user_df=user_df[['Mal ID']]

In [ ]:
user_df.shape

(3294, 1)

In [ ]:
user_df['Mal ID'] = user_df['Mal ID'].astype(float)
score_df['user_id'] = score_df['user_id'].astype(float)

score_df = score_df[score_df['user_id'].isin(user_df['Mal ID'])]

In [ ]:
rating_matrix=score_df.pivot_table(index='anime_id',columns='user_id',values='rating')
rating_matrix = rating_matrix.fillna(0)
rating_matrix.shape

(14364, 2268)

In [ ]:
from scipy.sparse import csr_matrix
sparse_rating_matrix = csr_matrix(rating_matrix)

from sklearn.neighbors import NearestNeighbors
# Train the NearestNeighbors model on the sparse rating matrix
nn_model = NearestNeighbors(metric='cosine', algorithm='auto')
nn_model.fit(sparse_rating_matrix)

# Create a mapping of anime IDs to their row indices in the sparse matrix
anime_index_map = {anime_id: idx for idx, anime_id in enumerate(rating_matrix.index)}

In [ ]:
def recommend_animes(anime_id, k, sparse_rating_matrix, nn_model, anime_index_map):
    if anime_id not in anime_index_map:
        raise ValueError(f"Anime ID {anime_id} not found in the rating matrix.")

    anime_index = anime_index_map[anime_id]

    anime_vector = sparse_rating_matrix[anime_index]

    distances, indices = nn_model.kneighbors(anime_vector, n_neighbors=k+1)  # k+1
    index_to_anime_id = {v: k for k, v in anime_index_map.items()}  # Reverse the mapping
    recommended_animes = [index_to_anime_id[idx] for idx in indices.flatten() if index_to_anime_id[idx] != anime_id][:k]

    return recommended_animes


In [ ]:
anime_name=input('Enter the Anime Name Here:')
k=int(input('Enter the no. of recommendations you want:'))

recommended_anime_ids = recommend_animes(anime_id=get_anime_id_by_name(anime_name,anime_df), k=k,sparse_rating_matrix=sparse_rating_matrix,nn_model=nn_model,anime_index_map=anime_index_map)

print(f"Top {k} recommended anime IDs for anime ID :",anime_name)
print_anime_rows_by_id(recommended_anime_ids)

Enter the Anime Name Here:Shingeki no Kyojin
Enter the no. of recommendations you want:10
Top 10 recommended anime IDs for anime ID : Shingeki no Kyojin
      Unnamed: 0  anime_id                         Name  \
3126        3126    4224.0                    Toradora!   
4917        4917    9253.0                  Steins;Gate   
5455        5455   10620.0             Mirai Nikki (TV)   
5746        5746   11757.0             Sword Art Online   
6108        6108   13601.0                  Psycho-Pass   
7156        7156   19815.0              No Game No Life   
7294        7294   20507.0                     Noragami   
7622        7622   22319.0                  Tokyo Ghoul   
8315        8315   25777.0  Shingeki no Kyojin Season 2   
9325        9325   30276.0                One Punch Man   

                  English name  Score                               Genres  \
3126                 Toradora!   8.08                       Drama, Romance   
4917               Steins;Gate   9.07    

In [ ]:
print(anime_index_map)

{1.0: 0, 5.0: 1, 6.0: 2, 7.0: 3, 8.0: 4, 15.0: 5, 16.0: 6, 17.0: 7, 18.0: 8, 19.0: 9, 20.0: 10, 21.0: 11, 22.0: 12, 23.0: 13, 24.0: 14, 25.0: 15, 26.0: 16, 27.0: 17, 28.0: 18, 29.0: 19, 30.0: 20, 31.0: 21, 32.0: 22, 33.0: 23, 43.0: 24, 44.0: 25, 45.0: 26, 46.0: 27, 47.0: 28, 48.0: 29, 49.0: 30, 50.0: 31, 51.0: 32, 52.0: 33, 53.0: 34, 54.0: 35, 55.0: 36, 56.0: 37, 57.0: 38, 58.0: 39, 59.0: 40, 60.0: 41, 61.0: 42, 62.0: 43, 63.0: 44, 64.0: 45, 65.0: 46, 66.0: 47, 67.0: 48, 68.0: 49, 69.0: 50, 71.0: 51, 72.0: 52, 73.0: 53, 74.0: 54, 75.0: 55, 76.0: 56, 77.0: 57, 79.0: 58, 80.0: 59, 81.0: 60, 82.0: 61, 83.0: 62, 84.0: 63, 85.0: 64, 86.0: 65, 87.0: 66, 88.0: 67, 89.0: 68, 90.0: 69, 91.0: 70, 92.0: 71, 93.0: 72, 94.0: 73, 95.0: 74, 96.0: 75, 97.0: 76, 98.0: 77, 99.0: 78, 100.0: 79, 101.0: 80, 102.0: 81, 103.0: 82, 104.0: 83, 105.0: 84, 106.0: 85, 107.0: 86, 108.0: 87, 109.0: 88, 110.0: 89, 111.0: 90, 112.0: 91, 113.0: 92, 114.0: 93, 115.0: 94, 116.0: 95, 117.0: 96, 118.0: 97, 119.0: 98, 120.

In [ ]:
import pickle
import joblib
from scipy.sparse import save_npz

# Paths to store the files
sparse_matrix_path = '/content/drive/MyDrive/Colab Notebooks/Machine Learning/Anime Recommendation Engine/Pickle Files/sparse_rating_matrix.npz'
nn_model_path = '/content/drive/MyDrive/Colab Notebooks/Machine Learning/Anime Recommendation Engine/Pickle Files/sparse_colab_based_nn_model.pkl'
index_map_path = '/content/drive/MyDrive/Colab Notebooks/Machine Learning/Anime Recommendation Engine/Pickle Files/anime_index_map.pkl'

# the sparse matrix
save_npz(sparse_matrix_path, sparse_rating_matrix)

joblib.dump(nn_model, nn_model_path, compress=3)  # Compress level 1-9

with open(index_map_path, 'wb') as f:
    pickle.dump(anime_index_map, f, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
import sklearn

In [ ]:
import scipy

In [ ]:
print('Pandas:',pd.__version__)
print('Numpy:',np.__version__)
print('Scikit Learn:',sklearn.__version__)
print('Pickle:',pickle.format_version)
print('Scipy:',scipy.__version__)
print('Joblib:',joblib.__version__)

Pandas: 2.2.2
Numpy: 1.26.4
Scikit Learn: 1.5.2
Pickle: 4.0
Scipy: 1.13.1
Joblib: 1.4.2
